# Notebook 03 — Rule-Based Backtesting (V1)

This notebook runs a simple, transparent backtest on the engineered features
stored in `screener_features`.

We focus on a basic question:

> Do days with **high volatility regime** and **hot/active edge buckets**
> lead to better next-day returns than ordinary days?

Steps:

1. Build a table with next-day returns per (run_date, ticker)  
2. Define a simple long-only signal using `vol_regime`, `edge_bucket`, `liquidity_bucket`  
3. Compare average next-day return for signal vs non-signal days  
4. (Optional) Plot a naive equity curve for “always take the signal”

In [1]:
import duckdb
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (10, 6)

# Connect to the same DuckDB file as the other notebooks
con = duckdb.connect("../data/volatility_alpha.duckdb")

con.sql("SHOW TABLES;").df()

,name
0,screener_features
1,screener_snapshots


## 1. Build (date, ticker) table with next-day returns

We compute the next-day percentage return for each (run_date, ticker) pair
using `last_price` from `screener_features`.

In [2]:
con.sql("""
CREATE OR REPLACE TABLE screener_returns AS
SELECT
    run_date,
    ticker,
    last_price,
    edge_score,
    vol_regime,
    edge_bucket,
    liquidity_bucket,
    LEAD(last_price) OVER (
        PARTITION BY ticker
        ORDER BY run_date
    ) AS next_last_price
FROM screener_features
ORDER BY ticker, run_date;
""")

con.sql("SELECT * FROM screener_returns LIMIT 10;").df()

,run_date,ticker,last_price,edge_score,vol_regime,edge_bucket,liquidity_bucket,next_last_price
0,2025-11-30,AMD,217.529999,35.113664,high,hot,thick,NaN
1,2025-11-30,NVDA,177.000000,21.891077,high,active,thick,NaN
2,2025-11-30,QQQ,619.250000,11.156231,normal,active,thick,NaN
3,2025-11-30,SPY,683.390015,7.770965,low,quiet,thick,NaN
4,2025-11-30,TSLA,430.170013,27.107530,high,hot,thick,NaN


## 2. Add next-day return target

We convert `last_price` and `next_last_price` into a simple % return.

In [3]:
con.sql("""
CREATE OR REPLACE TABLE screener_returns_with_target AS
SELECT
    *,
    CASE
        WHEN next_last_price IS NULL OR next_last_price = 0 THEN NULL
        ELSE (next_last_price / last_price - 1.0) * 100.0
    END AS next_day_return_pct
FROM screener_returns;
""")

sample = con.sql("""
    SELECT
        run_date,
        ticker,
        last_price,
        next_last_price,
        next_day_return_pct
    FROM screener_returns_with_target
    ORDER BY run_date DESC, ticker
    LIMIT 10;
""").df()

sample

,run_date,ticker,last_price,next_last_price,next_day_return_pct
0,2025-11-30,AMD,217.529999,NaN,NaN
1,2025-11-30,NVDA,177.000000,NaN,NaN
2,2025-11-30,QQQ,619.250000,NaN,NaN
3,2025-11-30,SPY,683.390015,NaN,NaN
4,2025-11-30,TSLA,430.170013,NaN,NaN


## 3. Define a simple long-only signal

V1 signal rule:

- Take a long signal when:
  - `vol_regime = 'high'`
  - `edge_bucket IN ('active', 'hot')`
  - `liquidity_bucket NOT IN ('thin', 'unknown')`

We hold for one day and evaluate `next_day_return_pct`.

In [4]:
con.sql("""
CREATE OR REPLACE TABLE screener_signals AS
SELECT
    *,
    CASE
        WHEN vol_regime = 'high'
             AND edge_bucket IN ('active', 'hot')
             AND liquidity_bucket NOT IN ('thin', 'unknown')
        THEN 1
        ELSE 0
    END AS signal_long
FROM screener_returns_with_target;
""")

con.sql("""
    SELECT
        run_date,
        ticker,
        vol_regime,
        edge_bucket,
        liquidity_bucket,
        signal_long,
        next_day_return_pct
    FROM screener_signals
    ORDER BY run_date DESC, ticker
    LIMIT 15;
""").df()

,run_date,ticker,vol_regime,edge_bucket,liquidity_bucket,signal_long,next_day_return_pct
0,2025-11-30,AMD,high,hot,thick,1,NaN
1,2025-11-30,NVDA,high,active,thick,1,NaN
2,2025-11-30,QQQ,normal,active,thick,0,NaN
3,2025-11-30,SPY,low,quiet,thick,0,NaN
4,2025-11-30,TSLA,high,hot,thick,1,NaN


## 4. Summary: signal vs non-signal days

We compare average next-day returns on days where the signal fires (1)
vs days where it does not (0).

In [5]:
summary = con.sql("""
    SELECT
        signal_long,
        COUNT(*) AS n,
        AVG(next_day_return_pct) AS avg_next_day_return,
        STDDEV(next_day_return_pct) AS std_next_day_return
    FROM screener_signals
    WHERE next_day_return_pct IS NOT NULL
    GROUP BY signal_long
    ORDER BY signal_long DESC;
""").df()

summary

,signal_long,n,avg_next_day_return,std_next_day_return


**Reading this table:**

- `signal_long = 1` → days where our rule fired  
- `signal_long = 0` → all other days  

With more data, we want to see:

- higher `avg_next_day_return` for `signal_long = 1`  
- a reasonable sample size `n` so results are stable

## 5. Naive equity curve for the signal-only strategy

We build a simple cumulative PnL stream:

- Filter to rows where `signal_long = 1` and we have a next-day return  
- Sort by date and accumulate the returns

This is not a production backtest, just a sanity check on how the signal behaves over time.

In [6]:
signal_df = con.sql("""
    SELECT
        run_date,
        ticker,
        signal_long,
        next_day_return_pct
    FROM screener_signals
    WHERE next_day_return_pct IS NOT NULL
    ORDER BY run_date, ticker;
""").df()

signal_trades = signal_df[signal_df["signal_long"] == 1].copy()

if not signal_trades.empty:
    signal_trades = signal_trades.sort_values("run_date")
    signal_trades["cum_return_pct"] = signal_trades["next_day_return_pct"].cumsum()

    plt.plot(signal_trades["run_date"], signal_trades["cum_return_pct"])
    plt.title("Naive Cumulative Return – Signal-Only Strategy")
    plt.xlabel("Date")
    plt.ylabel("Cumulative Return (%)")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
else:
    print("No trades found for signal_long = 1 yet.")

No trades found for signal_long = 1 yet.


## 6. Close connection

In [7]:
con.close()